In [1]:
from trl import SFTConfig

from src.callbacks import ClearMLCallback, SaveCustomWeightsCallback
from src.experiment import SFTExperiment
from src.trainer import VectorSFTTrainer


config = "configs/vector_sft/llama3_2_3b_custom.yaml"
experiment = SFTExperiment(config)
experiment.setup_lora_and_auxiliary()
data_collator = experiment.prepare_datasets()

training_args = SFTConfig(**experiment.cfg.trainer)

ClearML Task: created new task id=fca63f4f26164c13baad764bd7feb129
2025-07-05 19:41:17,011 - clearml.Task - INFO - Storing jupyter notebook directly as code
ClearML results page: https://app.clear.ml/projects/f9b3e11f9be94598b07489d1ae53b4c5/experiments/fca63f4f26164c13baad764bd7feb129/output/log


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

ClearML Monitor: Could not detect iteration reporting, falling back to iterations as seconds-from-start


In [2]:
self = experiment.model

In [6]:
inputs = data_collator(experiment.train_dataset.select(range(10)))


In [8]:
layer = self.get_layer(25)
        
# Get the original transformers model from the PeftModel wrapper. It contains the LoRA layers.
modified_base_model = self.base_model.get_base_model() # debug_model_structure.py confirms that this contains lora

# # Call the "body" of the model (e.g., LlamaModel) to get the last hidden state
# # This call goes through the LoRA layers.
# base_model_output = modified_base_model.model(
#     input_ids=input_ids,
#     attention_mask=attention_mask,
#     return_dict=True,
# )

# k_layer_hidden_state = layer._captured_hidden
# last_hidden_state = base_model_output.last_hidden_state

# logits = self.lm_head(last_hidden_state)


In [13]:
self.base_model.base_model.model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128258, 3072)
    (layers): ModuleList(
      (0-27): 28 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): lora.Linear(
            (base_layer): Linear(in_features=3072, out_features=3072, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=3072, out_features=128, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=128, out_features=3072, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (lora_magnitude_vector): ModuleDict()
          )
          (k_proj): lora.Linear(
            (base_layer): Linear(in_features=3072, out_features=1024, bias=False)
            (lora_dropout): ModuleDict(
              (defau

In [14]:
llama_with_lora = self.base_model.get_base_model()

self.k_layer_hidden_state = None

def hook_fn(module, inp, out):
    self.k_layer_hidden_state = out[0]
                
llama_with_lora.model.layers[self.k].register_forward_hook(hook_fn)

In [19]:
self.k_layer_hidden_state.shape

torch.Size([1, 461, 3072])

In [17]:
base_model_output = modified_base_model.model(
    input_ids=inputs["input_ids"][:1].cuda(),
    attention_mask=inputs["attention_mask"][:1].cuda(),
    return_dict=True,
)